In [ ]:
print("dsadsadsa")

In [ ]:
import pandas as pd
from sklearn.utils import resample
from pathlib import Path

dataset_dir = Path("./data/raw")
dataset_dir.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
dataset_path = dataset_dir / "creditcard.csv"
df = pd.read_csv(dataset_path)
#df.head

In [ ]:
# Load the dataset (Ensure you have 'creditcard.csv' in your working directory)
df = pd.read_csv(dataset_path)

# Check class distribution
print("Original class distribution:")
print(df["Class"].value_counts())

# Separate majority and minority classes
df_majority = df[df["Class"] == 0]  # Non-fraudulent transactions
df_minority = df[df["Class"] == 1]  # Fraudulent transactions

# Apply undersampling: Reduce majority class to match minority class size
df_majority_undersampled = resample(df_majority, 
                                    replace=False,    # Sample without replacement
                                    n_samples=len(df_minority),  # Match minority class size
                                    random_state=42)  # Reproducibility

# Combine minority class with undersampled majority class
df_undersampled = pd.concat([df_majority_undersampled, df_minority])

# Check new class distribution
print("\nNew class distribution after undersampling:")
print(df_undersampled["Class"].value_counts())

# Save the new dataset
#new_dataset_path = dataset_dir / "creditcard_undersampled.csv"
#df_undersampled.to_csv(dataset_path, index=False)
print("\nUndersampled dataset saved as 'creditcard_undersampled.csv'.")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
y = df_undersampled["Class"]
y=y.to_numpy()
temp_df=df_undersampled
temp_df=temp_df.drop(columns=["Class"])
column_names=temp_df.columns
X = df_undersampled.drop(columns=["Class"]).to_numpy()


#Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
#X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled,y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
# Convert to DataFrame for saving
train_df = pd.DataFrame(X_train, columns=column_names)

train_df["Class"] = y_train.astype("float")
val_df = pd.DataFrame(X_val, columns=column_names)
val_df["Class"] = y_val.astype("float")
test_df = pd.DataFrame(X_test, columns=column_names)
test_df["Class"] = y_test.astype("float")

#logger.info("Preprocessed data and split into train, validation, and test sets.")



In [ ]:
import boto3

# MinIO credentials and endpoint
endpoint_url = "http://localhost:9000"  # Adjust if running remotely
aws_access_key_id = "user"
aws_secret_access_key = "WW4e69Wwcv0w"
bucket_name = "models-bucket"
file_path = "userprefix.json"
object_name = "uploaded_example.txt"

# Create S3 client
s3_client = boto3.client(
    "s3",
    endpoint_url=endpoint_url,  # MinIO URL
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Upload file
s3_client.upload_file(file_path, bucket_name, object_name)

print(f"File {file_path} uploaded to {bucket_name} as {object_name}")

## MLFLOW

In [ ]:
import mlflow
import os
"""Configure MLflow tracking."""
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5000'
# Set the MLflow tracking username who is created this work
os.environ['MLFLOW_TRACKING_USERNAME'] = "user"
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name("creditcard_fraud_detection12321")
experiment_id=None
if experiment is None:
    experiment = client.create_experiment("creditcard_fraud_detectio12321")
    experiment_id=experiment.experiment_id
else:
    experiment_id = experiment.experiment_id

mlflow.set_experiment(experiment_id=experiment_id)



In [ ]:

import mlflow
import os


"""Configure MLflow tracking."""
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5000'  # Adjust for remote
os.environ['MLFLOW_TRACKING_USERNAME'] = "user"
client = mlflow.tracking.MlflowClient()
exp_name="Creditcard-Fraud-Detection"
# Check if experiment already exists
experiment = client.get_experiment_by_name(exp_name)
if experiment is None:
    experiment_id = client.create_experiment(exp_name)
    
else:
    experiment_id = experiment.experiment_id

mlflow.set_experiment(experiment_id=experiment_id)
print(f"MLflow experiment set up: id {experiment_id}")
mlflow.log_metric("Test size", 50)


In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("my_experiment_regression")

import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

# Generate dummy data
X, y = make_regression(n_samples=100, n_features=2, noise=0.1)
model = LinearRegression().fit(X, y)

# Start a new MLflow run
with mlflow.start_run():
    mlflow.sklearn.log_model(model, "model")
    mlflow.log_param("coef_", model.coef_.tolist())
    mlflow.log_metric("score", model.score(X, y))

2025/05/16 22:48:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run caring-mare-838 at: http://localhost:5000/#/experiments/2/runs/f4a1ef93fa7e42abbfcac77445402efb
🧪 View experiment at: http://localhost:5000/#/experiments/2
